<div >
<img src = "../banner.jpg" />
</div>

# Superlearners 

In [ ]:
require("pacman")
p_load("rmutil")
p_load("tidyverse")
p_load("nnls")
p_load("earth")
p_load("gam")
p_load("data.table")
p_load("SuperLearner")

# set the seed for reproducibility
set.seed(123)
# generate the observed data
n = 1000
x = runif(n, 0, 8)
y = 5 + 4 * sqrt(9 * x) * as.numeric(x <
    2) + as.numeric(x >= 2) * (abs(x - 6)^(2)) +
    rlaplace(n)
D <- data.frame(x, y) # observed data


In [ ]:
xl <- seq(0, 8, 0.1)
yl <- 5 + 4 * sqrt(9 * xl) * as.numeric(xl <
    2) + as.numeric(xl >= 2) * (abs(xl -
    6)^(2))

Dl <- data.frame(xl, yl) # for plotting the true dose-respon


In [ ]:
ggplot(Dl) +
  geom_point(aes(x=xl,y=yl))+
  theme_bw()

In [ ]:
ggplot(D) +
  geom_point(aes(x=x,y=y))+
  theme_bw()

In [ ]:
folds = 5
## split data into 5 groups for 5-fold ## cross-validation we do this here so ## that the exact same folds will be ## used in both the SL fit with the R ## package, and the hand coded SL 
index <- split(1:1000, 1:folds)
splt <- lapply(1:folds, function(ind) D[index[[ind]], ])
# view the first 6 observations in the # first [[1]] and second [[2]] folds 
head(splt[[1]])

In [ ]:
head(splt[[2]])

In [ ]:
# Fit using the SuperLearner Package

In [ ]:


SL.gam.5 <- create.Learner("SL.gam", params = list(deg.gam =5))

# Specifying the SuperLearner library # of candidate algorithms
sl.lib <- c(SL.gam.5$names, "SL.earth")


set.seed(123)
fitY <- SuperLearner(Y = y, X = data.frame(x),
    method = "method.NNLS", SL.library  = sl.lib,
    cvControl = list(V = folds, validRows = index))

# View the output: 'Risk' column returns the CV-MSE estimates
# 'Coef' column gives the weights # for the final SuperLearner

fitY


In [ ]:
# Now predict the outcome for all

yS <- predict(fitY, newdata = data.frame(x = xl), onlySL = T)$pred


Dl1 <- data.frame(xl, yS)

# Hand-coding Super Learner


In [ ]:
m1 <- lapply(1:folds, function(ii) gam(y ~s(x, 5), family = "gaussian", data = rbindlist(splt[-ii]))) 

In [ ]:
m2 <- lapply(1:folds, function(ii) earth(y ~
    x, data = rbindlist(splt[-ii]), degree = 2,
    penalty = 3, nk = 21, pmethod = "backward",
    nfold = 0, ncross = 1, minspan = 0, endspan = 0))

In [ ]:
p1 <- lapply(1:folds, function(ii) predict(m1[[ii]], newdata = rbindlist(splt[ii]), type = "response"))
p2 <- lapply(1:folds, function(ii) predict(m2[[ii]], newdata = rbindlist(splt[ii]), type = "response"))
# add the predictions to grouped # dataset 'splt'
for (i in 1:folds) {
    splt[[i]] <- cbind(splt[[i]], p1[[i]],
        p2[[i]])
}

In [ ]:
# Risks
risk1 <- lapply(1:folds, function(ii) mean((splt[[ii]][,
    2] - splt[[ii]][, 3])^2))
risk2 <- lapply(1:folds, function(ii) mean((splt[[ii]][, 2] - splt[[ii]][, 4])^2))

                
## 3: average the estimated risks across the 5 folds to obtain 1

a <- rbind(cbind("gam", mean(do.call(rbind,
    risk1), na.rm = T)), cbind("earth", mean(do.call(rbind,
    risk2), na.rm = T)))
a

In [ ]:
# checking to see match with SL output
fitY

In [ ]:
X <- data.frame(do.call(rbind, splt))[, -1] 
names(X) <- c("y", "gam", "earth")
head(X)

In [ ]:
SL.r <- nnls(cbind(X[, 2], X[, 3]), X[, 1])$x
alpha <- as.matrix(SL.r/sum(SL.r))
round(alpha, 3)

In [ ]:
fitY

In [ ]:
m1 <- gam(y ~ s(x, 5), family = "gaussian", data = D)
m2 <- earth(y ~ x, data = D, degree = 2, penalty = 3, nk = 21, pmethod = "backward", nfold = 0, ncross = 1, minspan = 0, endspan = 0)



## SL function
p1 <- predict(m1, newdata = data.frame(x = xl), type = "response")
p2 <- predict(m2, newdata = data.frame(x = xl), type = "response")
predictions <- cbind(p1, p2)
yS2 <- predictions %*% alpha

In [ ]:
Dl2 <- data.frame(xl, yS2,yS)
head(Dl2)